In [1]:
import pandas as pd

In [2]:
dataframe1 = pd.read_csv("/content/data.txt")

In [3]:
dataframe1.to_csv('data.csv', index = None)

In [4]:
df = pd.read_csv('data.csv',sep='\t')

In [5]:
df.head()

,CLASS,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,C,-2.254413,-0.591009,2.821589,5.015993,3.948079,-1.292129,0.022318,2.059526,-4.410204,1.356318,0.749807,29.043,33.862417,-1.868992
1,C,2.821589,-1.292129,0.022318,5.015993,1.356318,3.948079,0.749807,-2.254413,2.059526,-0.591009,-4.410204,23.490,26.140000,-2.303313
2,C,-4.410204,0.022318,2.059526,5.015993,-0.591009,0.749807,-2.254413,-1.292129,1.356318,3.948079,2.821589,29.340,31.674167,-2.245873
3,C,3.948079,-4.410204,-1.292129,2.821589,5.015993,2.059526,0.749807,-0.591009,-2.254413,0.022318,1.356318,36.160,34.899167,-2.559671
4,C,-2.254413,0.022318,0.749807,5.015993,3.948079,-1.292129,2.059526,1.356318,-4.410204,-0.591009,2.821589,27.490,30.772500,-2.071950


In [6]:
df = df.replace({'C': 1, 'H': -1})

In [7]:
df.head()

,CLASS,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,1,-2.254413,-0.591009,2.821589,5.015993,3.948079,-1.292129,0.022318,2.059526,-4.410204,1.356318,0.749807,29.043,33.862417,-1.868992
1,1,2.821589,-1.292129,0.022318,5.015993,1.356318,3.948079,0.749807,-2.254413,2.059526,-0.591009,-4.410204,23.490,26.140000,-2.303313
2,1,-4.410204,0.022318,2.059526,5.015993,-0.591009,0.749807,-2.254413,-1.292129,1.356318,3.948079,2.821589,29.340,31.674167,-2.245873
3,1,3.948079,-4.410204,-1.292129,2.821589,5.015993,2.059526,0.749807,-0.591009,-2.254413,0.022318,1.356318,36.160,34.899167,-2.559671
4,1,-2.254413,0.022318,0.749807,5.015993,3.948079,-1.292129,2.059526,1.356318,-4.410204,-0.591009,2.821589,27.490,30.772500,-2.071950


In [8]:
df['CLASS'].value_counts()

 1    60
-1    33
Name: CLASS, dtype: int64

In [9]:
pip install gplearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import math
from gplearn.genetic import SymbolicRegressor
import sklearn
from sympy import *
from sympy import symbols, Eq, solve
from sklearn.utils.random import check_random_state
import graphviz
import time
from gplearn.genetic import SymbolicClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [11]:
# create loocv procedure
cv = LeaveOneOut()

In [12]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [13]:
converter = {
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x*y,
    'div': lambda x, y : x/y,
    'sqrt': lambda x : x**0.5,
    'log': lambda x : log(x),
    'abs': lambda x : abs(x),
    'neg': lambda x : -x,
    'inv': lambda x : 1/x,
    'max': lambda x, y : max(x, y),
    'min': lambda x, y : min(x, y),
    'sin': lambda x : sin(x),
    'cos': lambda x : cos(x),
    'pow': lambda x, y : x**y,
}

In [15]:
# enumerate splits
y_true, y_pred, equation, res = list(), list(), list(), list()
feature=np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N'])
function_set = ['add', 'sub', 'mul']
X1 = np.array(X)
y1 = np.array(y)
errors=0
for train_ix, test_ix in cv.split(X):
  # split data
  # print(errors)
  try:
    # print(X1[train_ix, :].shape)
    # print(X1[test_ix, :].shape)
    # print(y1[train_ix].shape)
    # print(y1[test_ix].shape)
    X_train, X_test = X1[train_ix, :], X1[test_ix, :]
    y_train, y_test = y1[train_ix], y1[test_ix]
    # fit model
    model = SymbolicClassifier(population_size=1000, generations=30, tournament_size=40, stopping_criteria=0.1, const_range=(- 1.0, 1.0), init_depth=(2, 6), init_method='half and half', function_set=('add', 'sub', 'mul', 'div'), transformer='sigmoid', metric='log loss', parsimony_coefficient=0.001, p_crossover=0.9, p_subtree_mutation=0.01, p_hoist_mutation=0.01, p_point_mutation=0.01, p_point_replace=0.05, max_samples=1.0, class_weight=None, feature_names=feature, warm_start=False, low_memory=False, n_jobs=1, verbose=0, random_state=42)
    model.fit(X_train, y_train)
    # evaluate model
    y_gp1 = model.predict(X_test)
    # print(len(X_test[0]))
    # getting the equation
    eqn = sympify(str(model._program), locals=converter)
    eqn1 = str(eqn)
    # print(type(eqn1))
    # print(X_test)
    for i in range(0,len(X_test[0])):
      # d[chr(ord('A') + i)] = X_test[i] 
      # print(eqn1)
      # print(i)
      # print(chr(ord('A') + i),str(X_test[0][i]))
      eqn1 = eqn1.replace(chr(ord('A') + i),str(X_test[0][i]))
      
    # test_str = "A + J + 0.564"
    # for i in range(len(X_test[0])):
    #   test_str= test_str.replace(chr(ord('A') + i),str(X_test[0][i]))
    eqn1 = eqn1.replace('exp','math.exp')
    res1 = eval(eqn1)
    # print(res1)
    
    # print(X_test)
    # sol = solve(eqn)
    # print(sol)
    # print(eqn)
  except:
    errors+=1
    continue
  # store
  
  y_true.append(y_test[0])
  y_pred.append(y_gp1[0])
  equation.append(eqn)
  res.append(res1)

# calculate AUC-ROC
auc_score = roc_auc_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
# cm = confusion_matrix(y_test, y_gp1)
# sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
# specificity = cm[1,1]/(cm[1,0]+cm[1,1])
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)

print('roc_auc_score: ', auc_score)
print('F1 Score: ', f1)
print('Sensitivity: ', sensitivity)
print('Specificity : ', specificity)

roc_auc_score:  0.656060606060606
F1 Score:  0.7603305785123967
Sensitivity:  0.7666666666666667
Specificity :  0.5454545454545454


In [83]:
# from sklearn.metrics import classification_report
# print(classification_report(y_true,y_pred))

In [16]:
print(errors)

0


In [17]:
t = pd.DataFrame(y_true,columns=['y_true'])
t1 = pd.DataFrame(y_pred,columns=['y_pred'])

In [18]:
result = pd.concat([t, t1], axis=1, join='inner')
display(result)

,y_true,y_pred
0,1,-1
1,1,1
2,1,1
3,1,1
4,1,-1
...,...,...
88,1,1
89,1,1
90,-1,-1
91,-1,-1


In [19]:
t3 = pd.DataFrame(equation,columns=['GeneratedEquation'])

In [20]:
new_result = pd.concat([result,t3], axis=1, join='inner')

In [21]:
t4 = pd.DataFrame(res,columns=['EquationValues'])

In [22]:
final_result = pd.concat([new_result,t4], axis=1, join='inner')
display(final_result)

,y_true,y_pred,GeneratedEquation,EquationValues
0,1,-1,A + J + 0.564,-0.334095
1,1,1,A + J + 1 + 1.40383189955302*J/(G**2*L),3.167756
2,1,1,A + J + 1 + 1.40383189955302*J/(G**2*L),0.500707
3,1,1,A + J + 1 + 1.40383189955302*J/(G**2*L),4.971938
4,1,-1,A + J + 1 + 1.40383189955302*J/(G**2*L),-1.852538
...,...,...,...,...
88,1,1,A + J + 1 + 1.40383189955302*J/(G**2*L),5.419233
89,1,1,A + J + 1 + 1.40383189955302*J/(G**2*L),3.846227
90,-1,-1,A + J + 1 + 1.40383189955302*J/(G**2*L),-2.577667
91,-1,-1,A + J + 0.564,-0.940606


In [23]:
len(new_result['GeneratedEquation'].unique())

9

In [33]:
final_result['EquationValues'].describe()

count     93.000000
mean      -0.135100
std       22.194314
min     -198.975299
25%       -0.889416
50%        1.163197
75%        2.536208
max       71.632191
Name: EquationValues, dtype: float64

In [68]:
def x(lis):
  count=0
  pos=0
  for i in lis:
    if i<0:
      count+=1
    else:
      pos+=1
  print(count)
  print(pos)

In [69]:
x(y_pred)

32
61


In [109]:
### Converting predicted values into classes using new threshold
threshold=-3 # threshold kept as mean of the values got from equation
y_pred1 = np.array(res)
# print(y_pred1)
predicted_class1=np.ones(y_pred1.shape)
# print(predicted_class1)
predicted_class1[y_pred1<threshold]=-1
print(predicted_class1)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.]


In [110]:
y_pred == predicted_class1

array([False,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True, False, False,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True,  True, False, False,  True, False,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
       False, False, False])

In [111]:
# calculate AUC-ROC
auc_score = roc_auc_score(y_true, predicted_class1)
f1 = f1_score(y_true, predicted_class1)
# cm = confusion_matrix(y_test, y_gp1)
# sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
# specificity = cm[1,1]/(cm[1,0]+cm[1,1])
tn, fp, fn, tp = confusion_matrix(y_true, predicted_class1).ravel()
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)

print('roc_auc_score: ', auc_score)
print('F1 Score: ', f1)
print('Sensitivity: ', sensitivity)
print('Specificity : ', specificity)

roc_auc_score:  0.5068181818181817
F1 Score:  0.7814569536423841
Sensitivity:  0.9833333333333333
Specificity :  0.030303030303030304


In [24]:
new_result.to_csv('Extracted_data.csv')

Saving the Model

In [25]:
import pickle

In [26]:
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)